In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
train_ds = "../input/csc4851-homework4/birds_400/train"
test_ds = "../input/csc4851-homework4/birds_400/test"
val_ds = "../input/csc4851-homework4/birds_400/valid"

In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_ds,
                                                                     label_mode = "categorical",
                                                                     image_size = (224,224),
                                                                     batch_size = 32
                                                                     )
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_ds,
                                                                    label_mode = "categorical",
                                                                    image_size = (224,224),
                                                                    batch_size = 32
                                                                    )
val_data = tf.keras.preprocessing.image_dataset_from_directory(val_ds,
                                                                   label_mode= "categorical",
                                                                   image_size = (224,224),
                                                                   batch_size = 32
                                                                   )

In [ ]:
labels = train_data.class_names
plt.figure(figsize=(12,12))
for image, label in train_data.take(1):
  
  for i in range(9):
    
    plt.subplot(3,3, i+1)  
    plt.imshow(image[i].numpy().astype("uint8"))
    plt.title(labels[tf.argmax(label[i])])
    plt.axis("off")

In [ ]:
model = tf.keras.applications.EfficientNetB5(include_top= False, weights = "imagenet")

In [ ]:
from tensorflow.keras import layers

for layer in model.layers[:-5]:
    model.trainable = False


ip = tf.keras.Input(shape=(224,224,3))

mod = model(ip)
mod = layers.GlobalAveragePooling2D()(mod)

op = layers.Dense(400, activation="sigmoid")(mod)

model_2 = tf.keras.Model(ip,op)
model_2.summary()

In [ ]:
model_2.compile(
    loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

In [ ]:
model_2.fit(train_data,
              epochs = 5,
              steps_per_epoch = len(train_data),
              validation_data = val_data,
              validation_steps = len(val_data)
              )

In [ ]:
model_2_eval = model_2.evaluate(test_data)

In [ ]:
print(f"Model Accuracy: {model_2_eval[1]*100 : 0.2f}%")

In [ ]:
from keras.preprocessing import image
from keras import losses
from sklearn.metrics import log_loss
from glob import glob
import os
import csv 

In [ ]:
test_labels = test_data.class_names

len(test_labels)

In [ ]:
def prepare(img_path):
    img = image.load_img(img_path, target_size=(224,224))
    x = image.img_to_array(img)
    x = x/224
    return np.expand_dims(x, axis=0)
bce = losses.BinaryCrossentropy()

In [ ]:
template = [0]*400
log_loss = {}
test_images = glob("/kaggle/input/csc4851-homework4/birds_400/test/" + "/.jpg")

for path in test_images:
    result = model_2.predict([prepare(path)])
    actual_class = train_ds.split('/')[-2]
    actual_class = "BLACK & YELLOW  BROADBILL" if actual_class == "BLACK & YELLOW BROADBILL" else actual_class
    actual_index = classes.index(actual_class)
    template[actual_index] = 1
    log_loss_current = bce(template, result[0]).numpy()
    if actual_index in log_loss:
        log_loss[actual_index] += (log_loss_current)/100
    else:
        log_loss[actual_index] = (log_loss_current)/100
    template[actual_index] = 0

In [ ]:
import csv
id_bird = list(log_loss.keys())
values = list(log_loss.values())
abc = open('submission.csv', 'w')
writer = csv.writer(abc)
writer.writerow(['id','birds'])
for index in range(len(id_bird)):
    writer.writerow([ids[index],values[index]])
